In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import base64
import httplib2
import os

from PIL import Image
from PIL import ImageDraw

from googleapiclient import discovery
from oauth2client.client import GoogleCredentials

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'visio-for-kids.json'
DISCOVERY_URL='https://{api}.googleapis.com/$discovery/rest?version={apiVersion}'
credentials = GoogleCredentials.get_application_default()
service = discovery.build('vision', 'v1p4beta1', credentials=credentials,
                          discoveryServiceUrl=DISCOVERY_URL)

In [ ]:
# Load photos 
srcimage = "<FILE NAME>"

Image.open(srcimage)


In [ ]:
def get_labels(photo_file):
    with open(photo_file, 'rb') as image:
        image_content = base64.b64encode(image.read())
        service_request = service.images().annotate(body={
            'requests': [{
                'image': {
                    'content': image_content.decode('UTF-8')
                },
                'features': [{
                    'type': 'LABEL_DETECTION',
                    'maxResults': 35
                }]
            }]
        })
        response = service_request.execute()
        return response['responses'][0]['labelAnnotations']

In [ ]:
labels = get_labels(srcimage)
for label in labels:
    print ('%s : %2d%%' % (label['description'], label['score']*100))

In [ ]:
def get_logo(photo_file):
    with open(photo_file, 'rb') as image:
        image_content = base64.b64encode(image.read())
        service_request = service.images().annotate(body={
            'requests': [{
               'image': {
                        'content': image_content.decode('UTF-8')
                },
                'features': [{
                    'type': "LOGO_DETECTION",
                    'maxResults': 21
                }]
            }]
        })
        response = service_request.execute()
        return response['responses'][0]['logoAnnotations']

In [ ]:
logos = get_logo(srcimage)
for logo in logos:
    print ('%s : %2d%%' % (logo['description'], logo['score']*100))

In [ ]:
def highlight_logo(photo_file, responce):
    im = Image.open(photo_file)
    draw = ImageDraw.Draw(im)
    for logo in logos:
        if logo:
          color = '#00ff00'
        box = [(v.get('x', 0.0), v.get('y', 0.0)) for v in logo['boundingPoly']['vertices']]
        draw.line(box + [box[0]], width=5, fill=color)
    return im

In [ ]:
logos = get_logo(srcimage)
highlight_logo(srcimage, logos)

In [ ]:
def get_faces(photo_file):
    with open(photo_file, 'rb') as image:
        image_content = base64.b64encode(image.read())
        service_request = service.images().annotate(body={
            'requests': [{
                'image': {
                    'content':  image_content.decode('UTF-8')
                },
                'features': [{
                    'type': 'FACE_DETECTION',
                    'maxResults': 1000
                },]
            }]
        })
        response = service_request.execute()
        return response['responses'][0]['faceAnnotations']

In [ ]:
def highlight_faces(photo_file, responce):
    im = Image.open(photo_file)
    draw = ImageDraw.Draw(im)
    for face in faces:
        if face['joyLikelihood'] == 'VERY_LIKELY':
            color = '#ff8888'
        else:
            color = '#00ff00'
        box = [(v.get('x', 0.0), v.get('y', 0.0)) for v in face['fdBoundingPoly']['vertices']]
        draw.line(box + [box[0]], width=5, fill=color)
    return im

In [ ]:
faces = get_faces(srcimage)
highlight_faces(srcimage, faces)